## Imports

In [1]:
from text_adventure_games import (
    parsing, parsing, actions, things, blocks, viz
)

## Locations

In [3]:
camp = things.Location(
    "Camp",
    "You are standing in your tribe's base camp."
)
outbound_path = things.Location(
    "Outbound Path",
    "You are a winding path leaving the safety of camp. Ahead lies the Jungle.",
)
jungle = things.Location(
    "Jungle",
    "You are in the Jungle. There are vines and trees all around you."
)
well = things.Location(
    "Well",
    "You stand in front of a stone well. Get some water if you're thirsty.",
)
well.set_property("has_water", True)
challenge = things.Location(
    "Challenge",
    "You are at the Immunity Challenge! Compete for a chance to be safe at tribal."
)
cliffs = things.Location(
    "Cliffs",
    "You stand in front of the steep cliffs. Climb them carefully so you don't fall.",
)
beach = things.Location(
    "Beach",
    "You stand at the beach, toes in the sand. In front of you is the vast ocean."
)
ocean = things.Location(
    "Ocean",
    "You are at the edge of the ocean with waves washing up around your knees.",
)
bridge = things.Location(
    "Bridge",
    "You are on a rickety bridge. Be careful.",
)
bridge.set_property("is_broken", True)
tribal = things.Location(
    "Tribal",
    "You are at tribal counsel. Cast your vote to send someone home.",
)
death = things.Location(
    "The Afterlife",
    "You are dead. GAME OVER."
)
death.set_property("game_over", True)
quit = things.Location(
    "Wow, why did you even join this show if you were going to quit?",
    "You are out of the game; Jeff is seething. GAME OVER."
)
quit.set_property("game_over", True)

# Node connections
camp.add_connection("out", outbound_path)
outbound_path.add_connection("in", camp)
outbound_path.add_connection("north", jungle)
jungle.add_connection("west", jungle)
jungle.add_connection("north", well)
jungle.add_connection("east", challenge)
well.add_connection("south", jungle)
challenge.add_connection("west", jungle)
camp.add_connection("east", cliffs)
cliffs.add_connection("climb", beach)
cliffs.add_connection("jump", death)
beach.add_connection("south", ocean)
ocean.add_connection("north", beach)
beach.add_connection("east", bridge)
bridge.add_connection("west", beach)
bridge.add_connection("cross", tribal)
bridge.add_connection("run", death) # how do I remove this if bridge has been repaired
tribal.add_connection("west", bridge)
tribal.add_connection("quit", quit)

## Gettable Items

In [ ]:
machete = things.Item(
    "matchete",
    "a sturdy machete",
    "a sharp matchete capable of cutting anything."
)
machete.set_property("is_weapon", True)
machete.set_property("is_fragile", False)
camp.add_item(machete)

vines = things.Item(
    "vines",
    "long vines",
    "Weavable vines. These could be used to build a trap!",  
)
jungle.add_item(vines)

coconut_shells = things.Item(
    "coconut shells",
    "halved coconut shells",
    "These shells would be useful for carrying water!"
)
beach.add_item(coconut_shells)

driftwood = things.Item(
    "driftwood",
    "stout pieces of driftwood",
    "These planks look like they could hold a lot of weight."
)
beach.add_item(driftwood)


## Non-gettable items

In [ ]:
ocean_water = things.Item(
    "the ocean",
    "vast ocean",
    "THERE ARE FISH IN THE OCEAN.",
)
ocean_water.set_property("gettable", False)
ocean_water.set_property("has_fish", True)
ocean_water.add_command_hint("catch fish")
ocean_water.add_command_hint("catch fish with basket")
ocean.add_item(ocean_water)

## Characters

In [ ]:
# Player
player = things.Character(
    name="The player",
    description="You are an avid fan on the adventure of a lifetime.",
    persona="I am on an adventure. I am charismatic",
)
player.set_property("is_hungry", True)

# Player's torch
torch = things.Item("torch", "a torch", "Jeff says this thing is important.")
torch.set_property("is_lightable", True)
torch.set_property("is_lit", False)
torch.add_command_hint("light lamp")
player.add_to_inventory(torch)

## Non-Player Characters

In [ ]:
# An ally in camp
ally = things.Character(
    name="ally",
    description="A nice ally",
    persona="I am on your side and trustworth. We're in this together",
)
ally.set_property("is_hungry", False)
ally.set_property("character_type", "ally")
camp.add_character(ally)

# An enemy in camp
enemy = things.Character(
    name="ally",
    description="A mean enemy",
    persona="I will never vote with you. I despise you.",
)
enemy.set_property("is_hungry", True)
enemy.set_property("character_type", "enemy")
camp.add_character(enemy)

# An enemy in camp
neutral = things.Character(
    name="neutral",
    description="A mean neutral",
    persona="I could go either way. Persuade me.",
)
neutral.set_property("is_hungry", True)
neutral.set_property("character_type", "neutral")
camp.add_character(neutral)

# Probsty
jeffe = things.Character(
    name='Jeff',
    description="Our fearless leader",
    persona="The biggest fan on earth. He loves this game more than anything."
)
tribal.add_character(jeffe)


In [ ]:
# Actions needed
# weave fish trap
# Collect water
# Compete in Challenge
# 
